In [1]:
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage, HumanMessage
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import os
import requests
load_dotenv()

True

In [13]:
groq_api_key=os.getenv("GORQ_API_KEY")
exchange_rate_api = os.getenv("EXCHANGE_RATE_API")
request_url = f"https://v6.exchangerate-api.com/v6/{exchange_rate_api}/pair/USD/BDT"

In [3]:
response = requests.get(request_url)
response.json()

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745798401,
 'time_last_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1745884801,
 'time_next_update_utc': 'Tue, 29 Apr 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'BDT',
 'conversion_rate': 121.4694}

## Create Two tool

In [16]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given and base currency and target currency
    """
    
    request_url = f"https://v6.exchangerate-api.com/v6/{exchange_rate_api}/pair/{base_currency}/{target_currency}"
    response = requests.get(request_url)
    
    return response.json()

@tool
def convert(base_currency: int, conversion_rate: float) -> float:
    """
    given a currency conversion rate this function calculate the target currency value from a given base currency value.
    """
    
    return base_currency*conversion_rate

## Checking

In [ ]:
# checking
get_conversion_factor.invoke(
    {
        "base_currency": "USD",
        "target_currency": "BDT"
    }
)

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745798401,
 'time_last_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1745884801,
 'time_next_update_utc': 'Tue, 29 Apr 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'BDT',
 'conversion_rate': 121.4694}

In [8]:
convert.invoke(
    {
        "base_currency": 100,
        "conversion_rate": 121.4694
    }
)

12146.939999999999

## Do the tool binding

In [14]:
llm = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct", groq_api_key=groq_api_key, temperature=0.4)

In [18]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])
llm_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024499358050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002449935A900>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', temperature=0.4, model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor between a given and base currency and target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'given a currency conversion rate this function calculate the target currency value from a given base currency value.', 'parameters': {'properties': {'base_currency': {'type': 'integer'}, 'conversion_rate': {'type': 'number'}}, 'required': ['bas

In [19]:
messages = [
    HumanMessage(content="What is the conversion factor between USD to BDT, and based on can you convert 15 USD to Bangladesh Taka.")
]

In [20]:
messages

[HumanMessage(content='What is the conversion factor between USD to BDT, and based on can you convert 15 USD to Bangladesh Taka.', additional_kwargs={}, response_metadata={})]

In [22]:
ai_messages = llm_with_tools.invoke(messages)
ai_messages

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5gfj', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "BDT"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_nwdr', 'function': {'arguments': '{"base_currency": 15, "conversion_rate": 110.25}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 335, 'total_tokens': 381, 'completion_time': 0.090089477, 'prompt_time': 0.010542848, 'queue_time': 0.048888388, 'total_time': 0.100632325}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-47a7030b-ccaf-4ff1-9db4-cfd00fcf3d08-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'BDT'}, 'id': 'call_5gfj', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency': 15, 'conversion_rate': 110.25}, 'id': 'c

In [ ]:

ai_messages.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'BDT'},
  'id': 'call_5gfj',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 15, 'conversion_rate': 110.25},
  'id': 'call_nwdr',
  'type': 'tool_call'}]

`Did you see the problem two tools are executed automatically where fist tools are execution is correct but second tools is incorrect because first tool value as the input of the second tool. But hare is the problem like llm automatically set a random value.`

**Now fixed this problem**

In [24]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated
## this two libre will be to solve this problem.

## ***Again define the tools***

In [27]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given and base currency and target currency
    """
    
    request_url = f"https://v6.exchangerate-api.com/v6/{exchange_rate_api}/pair/{base_currency}/{target_currency}"
    response = requests.get(request_url)
    
    return response.json()

@tool
def convert(base_currency: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:  ## This tile llm will not set the conversion rate value for this InjectedToolsArgs
    """
    given a currency conversion rate this function calculate the target currency value from a given base currency value.
    """
    
    return base_currency*conversion_rate

In [28]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])
llm_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024499358050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002449935A900>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', temperature=0.4, model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor between a given and base currency and target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'given a currency conversion rate this function calculate the target currency value from a given base currency value.', 'parameters': {'properties': {'base_currency': {'type': 'integer'}}, 'required': ['base_currency'], 'type': 'object'}}}]}, co

In [29]:
messages = [
    HumanMessage(content="What is the conversion factor between USD to BDT, and based on can you convert 15 USD to Bangladesh Taka.")
]

In [30]:
ai_messages = llm_with_tools.invoke(messages)
ai_messages.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'BDT'},
  'id': 'call_5dvw',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 15},
  'id': 'call_5qte',
  'type': 'tool_call'}]

In [31]:
messages.append(ai_messages)
messages

[HumanMessage(content='What is the conversion factor between USD to BDT, and based on can you convert 15 USD to Bangladesh Taka.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5dvw', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "BDT"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_5qte', 'function': {'arguments': '{"base_currency": 15}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 325, 'total_tokens': 362, 'completion_time': 0.072542154, 'prompt_time': 0.010478629, 'queue_time': 0.049198531000000004, 'total_time': 0.083020783}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3a30e5d6-e109-4b83-a62e-5d948dd86841-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'U

## Now time to execute this tool calls

In [37]:
import json

for tool_call in ai_messages.tool_calls:
    # execute the 1st tool and get value of conversion rate
    if tool_call['name'] == "get_conversion_factor":
        tool_messages_1 = get_conversion_factor.invoke(tool_call)
        # fetch the conversion rate
        conversion_rate = json.loads(tool_messages_1.content)['conversion_rate']
        # append this tool messages into the messages history
        messages.append(tool_messages_1)
    # execute the second tool using the conversion rate from the tool1
    
    if tool_call['name'] == "convert":
        # fetch the current arg
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_messages_2 = convert.invoke(tool_call)
        messages.append(tool_messages_2)

In [38]:
messages

[HumanMessage(content='What is the conversion factor between USD to BDT, and based on can you convert 15 USD to Bangladesh Taka.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5dvw', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "BDT"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_5qte', 'function': {'arguments': '{"base_currency": 15}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 325, 'total_tokens': 362, 'completion_time': 0.072542154, 'prompt_time': 0.010478629, 'queue_time': 0.049198531000000004, 'total_time': 0.083020783}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3a30e5d6-e109-4b83-a62e-5d948dd86841-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'U

In [39]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD to BDT is 121.4694. \n15 USD is equivalent to 1822.041 Bangladesh Taka.'

## More interesting messages is Its is not a Ai agent.

### How a agent work -- autonomously
### My system works -- manually

## ***Agent workflow***
1. User says: "convert 15usd to bdt
2. llm thinks: i don't know the rate, so first call the get conversion factor
3. tool result is 121.4694
4. llm look at the result, and think again: now i know the rate now i should call the convert with the 15 and 120.4694
5. tool result: 1822.041
6. llm summarizes result: The conversion factor between USD to BDT is 121.4694. \n15 USD is equivalent to 1822.041 Bangladesh Taka.